# Restore Punctuation with Hugging Face Transformers

## Fix grammar mistakes

### Load transcript

In [1]:
input_text = open('transcripts/transcribed_speech_hf_pipelines.txt','r').readlines()[0]

In [2]:
input_text

'estamos con marcelo ferre es gastro enterólogo cómo le va a doctor buenas noches unasa ver hoy charlaba con nuestros compañeros de trabajo y decía no pero murió de un cáncer y había otra enfermedad previa y cómo es estocuéntenos la enfermedad de croncque esla enfermedad de cron correcto fernando la enfermedad de cron es una enfermedad inflamatoria del intestino sque entra dentro del grupo de las enfermedades inflamatorias intestinales que son enfermedades crónicas son autoinmunes tienen una base genética son raras en el sentido del diagnóstico son poco frecuentes hay pacientes en todo el país empacientes con esta enfermedad nada más nada más en todo el país o al menos diagnosticados se diagnosticado y conocido sicon algún seguimiento por parte de sus médicos uno cada chabitantes puede tener esta enfermedad aunque no la diagnostiquemos tan frecuentemente y en realidadcuesta diagnosticarla en el sentido al no verla y al no estar en contacto con ellos el diagnóstico es de difícil acceso 

### Using `pyspellchecker`
- https://pyspellchecker.readthedocs.io/en/latest/index.html

In [3]:
from spellchecker import SpellChecker

spell_py = SpellChecker(language='es', distance=2)  # Spanish dictionary

In [4]:
processed_text = spell_py.correction(input_text)
print(processed_text)

estamos con marcelo ferre es gastro enterólogo cómo le va a doctor buenas noches unasa ver hoy charlaba con nuestros compañeros de trabajo y decía no pero murió de un cáncer y había otra enfermedad previa y cómo es estocuéntenos la enfermedad de croncque esla enfermedad de cron correcto fernando la enfermedad de cron es una enfermedad inflamatoria del intestino sque entra dentro del grupo de las enfermedades inflamatorias intestinales que son enfermedades crónicas son autoinmunes tienen una base genética son raras en el sentido del diagnóstico son poco frecuentes hay pacientes en todo el país empacientes con esta enfermedad nada más nada más en todo el país o al menos diagnosticados se diagnosticado y conocido sicon algún seguimiento por parte de sus médicos uno cada chabitantes puede tener esta enfermedad aunque no la diagnostiquemos tan frecuentemente y en realidadcuesta diagnosticarla en el sentido al no verla y al no estar en contacto con ellos el diagnóstico es de difícil acceso e

### Using `autocorrect`
- https://github.com/filyp/autocorrect

In [5]:
from autocorrect import Speller
spell_autocorrect = Speller(lang='es',only_replacements=True)

In [6]:
processed_text_b = spell_autocorrect(input_text)
print(processed_text_b)

estamos con marcelo ferre es gastro entomólogo cómo le va a doctor buenas noches usada ver hoy charlaba con nuestros compañeros de trabajo y decía no pero murió de un cáncer y había otra enfermedad previa y cómo es estocuéntenos la enfermedad de croncque esta enfermedad de cron correcto fernando la enfermedad de cron es una enfermedad inflamatoria del intestino ique entra dentro del grupo de las enfermedades inflamatorias intestinales que son enfermedades crónicas son autoinmunes tienen una base genética son raras en el sentido del diagnóstico son poco frecuentes hay pacientes en todo el país impacientes con esta enfermedad nada más nada más en todo el país o al menos diagnosticados se diagnosticado y conocido simon algún seguimiento por parte de sus médicos uno cada chabitantes puede tener esta enfermedad aunque no la diagnostiquemos tan frecuentemente y en realidadcuesta diagnosticaron en el sentido al no verla y al no estar en contacto con ellos el diagnóstico es de difícil acceso e

## Using Silero 
Source: https://colab.research.google.com/github/snakers4/silero-models/blob/master/examples_te.ipynb

In [7]:
import os
import yaml
import torch
from torch import package

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)

with open('latest_silero_models.yml', 'r') as yaml_file:
    models = yaml.load(yaml_file, Loader=yaml.SafeLoader)
model_conf = models.get('te_models').get('latest')

In [8]:
# see avaiable languages
available_languages = list(model_conf.get('languages'))
print(f'Available languages {available_languages}')

# and punctuation marks
available_punct = list(model_conf.get('punct'))
print(f'Available punctuation marks {available_punct}')

Available languages ['en', 'de', 'ru', 'es']
Available punctuation marks ['.', ',', '-', '!', '?', '—']


### With language label

In [9]:
model_url = model_conf.get('package')

model_dir = "downloaded_model"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, os.path.basename(model_url))

if not os.path.isfile(model_path):
    torch.hub.download_url_to_file(model_url,
                                   model_path,
                                   progress=True)

imp = package.PackageImporter(model_path)
model = imp.load_pickle("te_model", "model")

def apply_te(text, lan='es'):
    return model.enhance_text(text, lan)

In [10]:
output_text = apply_te(processed_text, lan='es')
print(output_text)

Estamos con Marcelo Ferre es gastro enterólogo cómo le va a doctor buenas noches Unasa ver hoy charlaba con nuestros compañeros de trabajo y decía no pero murió de un cáncer y había otra enfermedad previa y cómo es estocuéntenos. La enfermedad de Croncque esla enfermedad de Cron correcto Fernando. La enfermedad de Cron es una enfermedad inflamatoria del intestino Sque entra dentro del grupo de las enfermedades inflamatorias intestinales que son enfermedades crónicas son autoinmunes tienen una base genética. Son raras en el sentido del diagnóstico Son poco frecuentes. Hay pacientes en todo el país empacientes con esta enfermedad nada más nada más en todo el país o al menos diagnosticados. Se diagnosticado y conocido sicon algún seguimiento por parte de sus médicos. Uno cada chabitantes puede tener esta enfermedad, aunque no la diagnostiquemos tan frecuentemente y en realidadcuesta diagnosticarla en el sentido al no verla y al no. estar en contacto con ellos el diagnóstico es de difícil 

### With fasttext for language detection

In [11]:
from ftlangdetect import detect

lan = detect(text=processed_text, low_memory=False)['lang']
print(f"Detected language: {lan}")

Detected language: es


In [12]:
import torch

model, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                  model='silero_te')

Using cache found in /home/studio-lab-user/.cache/torch/hub/snakers4_silero-models_master


In [13]:
output_text = apply_te(processed_text, lan='es')
print(output_text)

Estamos con Marcelo Ferre es gastro enterólogo cómo le va a doctor buenas noches Unasa ver hoy charlaba con nuestros compañeros de trabajo y decía no pero murió de un cáncer y había otra enfermedad previa y cómo es estocuéntenos. La enfermedad de Croncque esla enfermedad de Cron correcto Fernando. La enfermedad de Cron es una enfermedad inflamatoria del intestino Sque entra dentro del grupo de las enfermedades inflamatorias intestinales que son enfermedades crónicas son autoinmunes tienen una base genética. Son raras en el sentido del diagnóstico Son poco frecuentes. Hay pacientes en todo el país empacientes con esta enfermedad nada más nada más en todo el país o al menos diagnosticados. Se diagnosticado y conocido sicon algún seguimiento por parte de sus médicos. Uno cada chabitantes puede tener esta enfermedad, aunque no la diagnostiquemos tan frecuentemente y en realidadcuesta diagnosticarla en el sentido al no verla y al no. estar en contacto con ellos el diagnóstico es de difícil 

## Using "Deep Multilingual Punctuation"

The `PunctuationModel` class an process texts of any length. Note that processing of very long texts can be time consuming.

In [14]:
from deepmultilingualpunctuation import PunctuationModel

In [15]:
model = PunctuationModel(model='oliverguhr/fullstop-punctuation-multilingual-base')

/home/studio-lab-user/.conda/envs/machinelearnear-asr-clinical-biomedical-ehr/lib/python3.9/site-packages/transformers/pipelines/token_classification.py:135: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.NONE"` instead.
  warnings.warn(


In [16]:
result = model.restore_punctuation(output_text)
print(result)

Estamos con Marcelo Ferre. es gastro enterólogo. cómo le va a doctor? buenas noches, Unasa ver. hoy charlaba con nuestros compañeros de trabajo y decía: no, pero murió de un cáncer y había otra enfermedad previa. y cómo es estocuéntenos La enfermedad de Croncque? esla enfermedad de Cron, correcto, Fernando? La enfermedad de Cron es una enfermedad inflamatoria del intestino Sque entra dentro del grupo de las enfermedades inflamatorias intestinales, que son enfermedades crónicas, son autoinmunes, tienen una base genética, Son raras en el sentido del diagnóstico, Son poco frecuentes. Hay pacientes en todo el país empacientes con esta enfermedad, nada más, nada más en todo el país, o al menos diagnosticados, Se diagnosticado y conocido, sicon algún seguimiento por parte de sus médicos. Uno cada chabitantes puede tener esta enfermedad, aunque no la diagnostiquemos tan frecuentemente y en realidadcuesta diagnosticarla en el sentido, al no verla y al no estar en contacto con ellos, el diagnós

## Save results

In [17]:
with open(f'transcripts/transcribed_speech_w_corrected_punctuation.txt', "w") as f:
    f.write(result)

## References

- https://huggingface.co/speeqo/bert-restore-punctuation
- https://huggingface.co/softcatala/fullstop-catalan-punctuation-prediction
- https://github.com/oliverguhr/fullstop-deep-punctuation-prediction
- https://huggingface.co/kredor/punctuate-all
- https://huggingface.co/SJ-Ray/Re-Punctuate
- https://github.com/xashru/punctuation-restoration
- https://github.com/snakers4/silero-models#text-enhancement